## Setup
#### Load the API key and relevant Python libaries.

In [13]:
import io
from dotenv import dotenv_values, load_dotenv, find_dotenv
import openai
from openai import OpenAI
import os
from copy import deepcopy
import json
import time
import ast
import re

import pprint

# LLM Open AI

In [14]:
# Get the first key from the uploaded dictionary
env_file_key = "../../auixiliary/env_HumanRights"

# Open the file and read its content
with open(env_file_key, 'r', encoding='utf-8') as file:
    env_content = file.read()

# Load the content into a variable
env_variables = dotenv_values(stream=io.StringIO(env_content))

api_key = env_variables['OPENAI_API_KEY']
# openai.api_key = api_key

client = OpenAI(
    # This is the default and can be omitted
    api_key=api_key,
)

# Models

In [15]:
def chat_gpt(prompt, temperature=0):
    response = client.chat.completions.create(
        model="gpt-4",
        messages=prompt,
        temperature=temperature 
    )
    return response.choices[0].message.content.strip()

# Iterate and Save Use Riskiness Results

# Functions

In [16]:
def replace_key(d, old_key, new_key):
  """
  Replace `old_key` with `new_key` in dictionary `d`.
  The associated value is retained.
  """
  if old_key in d:
      d[new_key] = d.pop(old_key)
  return d

## Read In Prompt Result

In [17]:
def read_prompt_output(file_path):

  # Read the uploaded file
  with open(file_path, 'r') as file:
      data = json.load(file)

  return data

# Read Risk Reports

In [18]:
risks_categorized = read_prompt_output("../../results/risk_categories/FULL_risk_categorisation.json")

# 1 MITIGATION PROMPT

In [22]:
# Assuming you have the variables domain, purpose, aiCapability, aiUser, and aiSubject defined with appropriate values

# V1:   AI Users:      In any case, the mitigation strategy that you propose should be for the oridinary individuals who are potential users of the AI System (and not for e.g., developers). What can they do in their power if anything to mitigate some of the risks of the AI System and have it useful?
# V2:   Ai Subjects:          In any case, the mitigation strategy that you propose should be for the oridinary individuals who are potential subjects of the AI System (and not for e.g., developers). What can they do in their power if anything to mitigate some of the risks of the AI System and have it still useful?


MESSAGES = [
    {
        'role': 'system',
        'content': """You are an experienced regulatory compliance specialists who works in the field of AI technology regulation. You are thoughtful, decisive, experienced and conscientious.
        You have access to the entirety of the EU AI Act and its amendments, which outline how various AI technologies are to be regulated and risk-classified within the European Union."""
    },
    {
        'role': 'user',
        'content': """You will be provided in input with an AI technology use description through five concepts:

        Domain: "{}",
        Purpose: "{}",
        Capability: "{}",
        AI User: "{}",
        AI Subject: "{}"

        along with a list of risks associated with this AI use:

        "Overall Risk Assessment":
        "{}"

        For each of the risks, conduct the following assessment in steps:
          Step 1: Think which of the AI stakeholders from the following list [AI User, AI Subject, Institutions and Environment, System Developer, Interaction Designer, Compliance Expert, and Organisational Leader] might be in the best position to mitigate or tackle the risks.
          Propose as many mitigation strategies as many AI stakeholders can mitigate the given risk. Think what can they do in their power if anything to mitigate the risk of the AI System and have the AI technology use still being useful?
          Step 2: Moreover, label the type of the mitigation strategy proposed considering the definitions of 4 possible labels below:
            * Avoidance: take measures to avoid the risk from occurring.
            * Reduction: reduce the likelihood of a risk happening or the impact should it occur.
            * Transference: pass the risk consequence to a third party.
            * Acceptance: accepting the risk as it stands (this means, no mitigation strategy was found).
          Step 3: Give very practical, actionable and concrete ideas on how the person(s) in this AI stakeholder role(s) can practically implement the proposed mitigation strategies. Provide a specific, actionable plan for how individuals in this AI stakeholder role can effectively carry out risk mitigation.
          Step 4: Summarize the mitigated version of the AI use and describe your reasoning for why its risk is now lowered. It is of utmost importance to exercise precision and make accurate judgments when classifying the risk associated with the AI system.

         Please return the mitigation strategies in the following format:
         {{
          "Mitigated Risks":
            [
            {{
              "Risk ID": "The exact copy-paste of the risk ID from input for which this mitigation strategy is defined",
              "Risk description for risk being mitigated": "The exact copy-paste of the risk from input for which this mitigation strategy is defined",
              "Mitigation Strategy": "The AI system can become low risk if ...",
              "Type of Mitigation Strategy": "[Avoidance/Reduction/Transference/Acceptance]",
              "AI Stakeholder for whom is the strategy": "[AI User/AI Subject/Institutions and Environment/System Developer/Interaction Designer/Compliance Expert/Organisational Leader]",
              "Practical mitigation actions": "Give 2-5 very practical, concise, actionable and concrete ideas how the person in this role can practically implement the mitigation against this specific risk.",
              "New System Description": "The mitigated AI system is intended to be used ..."
            }},
            {{
              "Risk ID": "The exact copy-paste of the risk ID from input for which this mitigation strategy is defined",
              "Risk description for risk being mitigated": "The exact copy-paste of the risk from input for which this mitigation strategy is defined",
              "Mitigation Strategy": "The AI system can become low risk if ...",
              "Type of Mitigation Strategy": "[Avoidance/Reduction/Transference/Acceptance]",
              "AI Stakeholder for whom is the strategy": "[AI User/AI Subject/Institutions and Environment/System Developer/Interaction Designer/Compliance Expert/Organisational Leader]",
              "Practical mitigation actions": "Give 2-5 very practical, concise, actionable and concrete ideas how the person in this role can practically implement the mitigation against this specific risk.",
              "New System Description": "The mitigated AI system is intended to be used ..."
            }},
            ...
            {{
              "Risk ID": "The exact copy-paste of the risk ID from input for which this mitigation strategy is defined",
              "Risk description for risk being mitigated": "The exact copy-paste of the risk from input for which this mitigation strategy is defined",
              "Mitigation Strategy": "The AI system can become low risk if ...",
              "Type of Mitigation Strategy": "[Avoidance/Reduction/Transference/Acceptance]",
              "AI Stakeholder for whom is the strategy": "[AI User/AI Subject/Institutions and Environment/System Developer/Interaction Designer/Compliance Expert/Organisational Leader]",
              "Practical mitigation actions": "Give 2-5 very practical, concise, actionable and concrete ideas how the person in this role can practically implement the mitigation against this specific risk.",
              "New System Description": "The mitigated AI system is intended to be used ..."
            }}
            ]
        }}

        ***Check your planned output before outputting it: if it contains any explanations besides the JSON string, omit the explanations. Make sure to output ONLY a correctly formatted JSON string and nothing else.***

        Please be aware that in certain scenarios, there may be an absence of risks related to this use. In such instances, you should disregard the missing risk and you should generate an output in the form of an empty JSON structure, as illustrated below:
          
          {{
            "Mitigated Risks":[]
          }}
        ***Check your planned output before outputting it: if it contains any explanations besides the JSON string, omit the explanations. Make sure to output ONLY a correctly formatted JSON string and nothing else.***


        """
    }
]



def format_prompt(MESSAGES, domain,purpose,aiCapability,aiUser,aiSubject,risks):
    S = "test {}"
    messages = deepcopy(MESSAGES)
    messages[1]['content'] = messages[1]['content'].format(domain,purpose,aiCapability,aiUser,aiSubject,risks)
    return messages


## 1 APPLY / RUN

In [85]:
# risks_categorized

In [23]:
# cost = 0

FULL_RES = []

start_time = time.time()
i = 0

found = 0

for useElements in risks_categorized:
  # print (useElements)
  useI = str(useElements['id'])

  if int(useI) == 100:
    found = 1

  if not found:
    continue

  print (f" Parsing use {useI}")

  # Variables for message placeholders
  domain = useElements['Details'][0]
  purpose = useElements['Details'][1]
  aiCapability = useElements['Details'][2]
  aiUser = useElements['Details'][3]
  aiSubject = useElements['Details'][4]

  risks = useElements['Risks']

  # Extracting "Use i" details
  use_i_details = [domain, purpose, aiCapability, aiUser, aiSubject]

  # print(use_i_details)

  # adapt the prompt for useI
  messages = format_prompt(MESSAGES, domain, purpose, aiCapability, aiUser, aiSubject, risks)

  # run the prompt
  response = chat_gpt(messages, temperature=0)
  print(response)

  # response, token_count = chat_gpt(messages, temperature=0)
  # res = token_count
  # cost_chunk = (res['prompt_tokens'] * 0.03  + res['completion_tokens'] * 0.06)/1000.0
  # cost += cost_chunk

  # print (response)
  response = ast.literal_eval(response)

  combined_response = {}
  combined_response["id"]= useI
  combined_response["Details"] = use_i_details
  combined_response["Risks"] = risks
  combined_response["Mitigated Risks"] = response["Mitigated Risks"]
  # for k, v in response.items():
  #   combined_response[k] = v
  print (combined_response)

  end_time = time.time()

  print(f"Execution time: {end_time - start_time:.5f} seconds")
  # print (f"TOTAL COST {cost}")

  FULL_RES.append(combined_response)

 Parsing use 100
{
  "Mitigated Risks":
    [
    {
      "Risk ID": "100-1",
      "Risk description for risk being mitigated": "Discriminates against certain groups if the facial recognition technology is biased or inaccurate.",
      "Mitigation Strategy": "The AI system can become low risk if the System Developer ensures the facial recognition technology is unbiased and accurate.",
      "Type of Mitigation Strategy": "Avoidance",
      "AI Stakeholder for whom is the strategy": "System Developer",
      "Practical mitigation actions": [
        "System Developer should use diverse datasets for training the AI system to ensure it can accurately recognize faces from different ethnicities, ages, and genders.",
        "System Developer should conduct regular audits and tests to ensure the AI system is not biased or inaccurate.",
        "System Developer should implement a feedback mechanism for AI Users and AI Subjects to report any issues or biases."
      ],
      "New System Desc

In [24]:
###############################
# save result
with open(f"../../results/mitigations/FULL_mitigations.json", "w") as json_file:
    json.dump(FULL_RES, json_file, indent=4)  # 4 spaces of indentation
# Download the file to your local machine

# THE END